# 1.Setup

Pull in respective libraries to prepare the notebook environment.

In [1]:
# !git clone --depth 1 https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

d:\android\COC_robot\model\yolov5
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import utils
display = utils.notebook_init()  # checks


YOLOv5  2025-3-30 Python-3.9.18 torch-2.2.0+cpu CPU


Setup complete  (16 CPUs, 31.9 GB RAM, 463.0/753.9 GB disk)


In [3]:
torch.cuda.is_available()  # True (GPU) or False (CPU)

False

# 2. 训练模型

In [4]:
import os
print("当前路径:", os.getcwd())

# --device cpu 使用cpu训练  
# --device 0 切换到GPU 0，如果GPU可用，请使用此选项
# batch-size 可调大，GPU显存足够大时可调大
# epochs 训练轮数可增加。 这里仅作演示epochs=10
# --weights 预训练权重 ../../util/best.pt 此模型为经过训练的模型，使用此模型进行微调训练
# hyp.game-ui.yaml 可根据调整优化
!python train.py --imgsz 640 --batch-size 1 --epochs 10 --data ../data.yaml --weights ../../util/best.pt --patience 0 --hyp ../hyp.game-ui.yaml

# 若模型需要识别新元素的话，需要新增数据dataset。
# 最终模型训练效果，最好需要精确率（Precision>0.95）和召回率（Recall>0.90）

# 训练输出结果 model\yolov5\runs

当前路径: d:\android\COC_robot\model\yolov5
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


train: weights=../../util/best.pt, cfg=, data=../data.yaml, hyp=../hyp.game-ui.yaml, epochs=10, batch_size=1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=20, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
YOLOv5  2025-3-30 Python-3.9.18 torch-2.2.0+cpu CPU

hyperparameters: lr0=0.001, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, 

# 3. 分析模型检测效果

In [7]:
!python val.py \
    --weights ./runs/train/exp/weights/best.pt \
    --data ../data.yaml \
    --task test \
    --save-json \
    --save-txt \
    --save-conf \
    --name recall_analysis
# 下面的训练输出结果显示 末尾部分标签准确率较低，假阳性偏高。可尝试增加对应的样本数量

loading annotations into memory...


val: data=../data.yaml, weights=['./runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=True, project=runs\val, name=recall_analysis, exist_ok=False, half=False, dnn=False
YOLOv5  2025-3-30 Python-3.9.18 torch-2.2.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7045186 parameters, 0 gradients, 15.9 GFLOPs

test: Scanning D:\android\COC_robot\model\dataset\labels...:   0%|          | 0/18 [00:00<?, ?it/s]
test: Scanning D:\android\COC_robot\model\dataset\labels... 1 images, 0 backgrounds, 0 corrupt:   6%|▌         | 1/18 [00:06<01:48,  6.39s/it]
test: Scanning D:\android\COC_robot\model\dataset\labels... 18 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18/18 [00:06<00:00,  2.79it/s]
test: WARNING  Cache directory D:\android\COC_robot\model\dataset is not writeable: [WinError 183] : 'D

In [8]:
!python detect.py --weights ./runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source ../dataset/images/

detect: weights=['./runs/train/exp/weights/best.pt'], source=../dataset/images/, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2025-3-30 Python-3.9.18 torch-2.2.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7045186 parameters, 0 gradients, 15.9 GFLOPs
image 1/18 D:\android\COC_robot\model\dataset\images\device_0.JPG: 288x640 4 golds, 3 adds, 1 donate, 1 replay, 78.3ms
image 2/18 D:\android\COC_robot\model\dataset\images\shot_1096.png: 288x640 1 close, 1 leilong, 1 shandian, 1 kuangbao, 54.9ms
image 3/18 D:\android\COC_robot\model\dataset\images\shot_2989.png: 384x640 1 close, 1 qiqiu, 1 leilong,

# 4. 对比仓库使用的模型效果

`util\results.csv` 为当前使用的模型训练结果，若准确度和召回率均优于该模型。建议直接回传到原始仓库。 

In [ ]:
# !python export.py --imgsz 1280 --weight runs/train/exp0/weights/best.pt --data ../data.yaml